In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re #정규식 지원

C:\Users\BIT\miniconda3\envs\ml-env\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
url='http://www.yes24.com/24/Category/Display/001001046001?FetchSize=40'
r = requests.get(url)
print(r)
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
print(soup)

In [ ]:
url ='http://www.yes24.com/24/Category/Display/001001046001?FetchSize=40&PageNumber=3'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
url_tag = 'a.bgYUI.ico_nWin'
book_url = soup.select(url_tag)
print(book_url)
print(len(book_url))

In [ ]:
linkList=[]
for i in range(len(book_url)):
    linkList.append(book_url[i].get('href'))
    
print(linkList)

1. 카테고리 페이지에서 각 도서별 url scraping

In [2]:
url = 'http://www.yes24.com/24/Category/Display/001001046001?FetchSize=40&PageNumber=1'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
url_tag = 'div.goods_name>a'
books = soup.select(url_tag)
print(books)
print(len(books))

[<a href="/Product/Goods/99308021">불편한 편의점</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/99308021" target="_blank"></a>, <a href="/Product/Goods/111088149">불편한 편의점 2</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/111088149" target="_blank"></a>, <a href="/Product/Goods/112253263">아버지의 해방일지</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/112253263" target="_blank"></a>, <a href="/Product/Goods/17320795">구의 증명</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/17320795" target="_blank"></a>, <a href="/Product/Goods/111085946">하얼빈</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/111085946" target="_blank"></a>, <a href="/Product/Goods/108887930">작별인사</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/108887930" target="_blank"></a>, <a href="/Product/Goods/113737429">이토록 평범한 미래</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/113737429" target="_blank"></a>, <a href="/Product/Goods/91065309">달러구트 꿈 백화점 (레인보우 에디션)</a>, <a class="bgYUI ico_nWin" href="/Product/Goods/9

In [3]:
url_list=[]
for book in books:
    sub_url = 'http://www.yes24.com/' + book.get('href')
    sub_soup = BeautifulSoup((requests.get(sub_url).text), 'lxml')
    url_list.append(sub_url)
            
print(url_list)
print(len(url_list))

['http://www.yes24.com//Product/Goods/99308021', 'http://www.yes24.com//Product/Goods/99308021', 'http://www.yes24.com//Product/Goods/111088149', 'http://www.yes24.com//Product/Goods/111088149', 'http://www.yes24.com//Product/Goods/112253263', 'http://www.yes24.com//Product/Goods/112253263', 'http://www.yes24.com//Product/Goods/17320795', 'http://www.yes24.com//Product/Goods/17320795', 'http://www.yes24.com//Product/Goods/111085946', 'http://www.yes24.com//Product/Goods/111085946', 'http://www.yes24.com//Product/Goods/108887930', 'http://www.yes24.com//Product/Goods/108887930', 'http://www.yes24.com//Product/Goods/113737429', 'http://www.yes24.com//Product/Goods/113737429', 'http://www.yes24.com//Product/Goods/91065309', 'http://www.yes24.com//Product/Goods/91065309', 'http://www.yes24.com//Product/Goods/106211628', 'http://www.yes24.com//Product/Goods/106211628', 'http://www.yes24.com//Product/Goods/72172961', 'http://www.yes24.com//Product/Goods/72172961', 'http://www.yes24.com//Prod

2. 책 세부페이지에서 데이터 scraping

In [4]:
title_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
author_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a' 
publisher_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a'
price_tag = 'em.yes_m'
isbn_tag = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td'
img_tag = 'img.gImg'
category_tag = '#infoset_goodsCate > div.infoSetCont_wrap > dl > dd > ul > li:nth-child(1) > a:nth-child(4)'

url ='http://www.yes24.com/Product/Goods/116208935'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
book_title = soup.select(title_tag)[0].text
book_author = soup.select(author_tag)[0].text
book_publisher = soup.select(publisher_tag)[0].text
book_price = int(re.sub('[^0-9]', '', soup.select(price_tag)[0].text)) #정규식 사용
book_isbn = soup.find('th', text='ISBN13').find_next_sibling('td').text
book_img = soup.select(img_tag)[0].get('src')
book_category = soup.select(category_tag)[0].text

print(book_title)
# print(book_author)
# print(book_publisher)
# print(book_price)
# print(book_isbn)
# print(book_img)
# print(book_category)

안젤리크 


3. 함수 생성

In [23]:
def yes24DataReader(CategoryNum, startPage, endPage):

    root_url = 'http://www.yes24.com'

    url_1 = 'http://www.yes24.com/24/Category/Display/'
    url_2 = '?PageNumber='
    url_set = url_1 + CategoryNum + url_2
    book_list=[]

    # 월 별 조회 시 최대 50쪽이지만, 간단한 설명을 위해 2쪽까지만 수집
    for i in range(startPage, endPage+1):

        url = url_set + str(i)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'lxml')
        url_tag = 'a.bgYUI.ico_nWin'
        books = soup.select(url_tag)

        # 수집 중인 페이지 번호 출력
        print('# Page', i)

        # 개별 도서 정보 수집
        for book in books:

            sub_url = root_url + book.get('href')
            sub_html = requests.get(sub_url).text
            sub_soup = BeautifulSoup(sub_html, 'lxml')            
#             print(sub_url)

            title_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
            author_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a' 
            publisher_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a'
            isbn_tag = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td'
            img_tag = 'img.gImg'
            category_tag = '#infoset_goodsCate > div.infoSetCont_wrap > dl > dd > ul > li:nth-child(1) > a:nth-child(4)'


            book_title = sub_soup.select(title_tag)[0].text.strip()
            book_author = sub_soup.select(author_tag)[0].text
            book_publisher = sub_soup.select(publisher_tag)[0].text
            try :
                book_isbn = sub_soup.find('th', text='ISBN13').find_next_sibling('td').text
            except :
                book_isbn = ''
                
            book_img = sub_soup.select(img_tag)[0].get('src')
            book_category = sub_soup.select(category_tag)[0].text

#             print(book_title)
#             print(book_author)
#             print(book_publisher)
#             print(book_price)
#             print(book_isbn)
#             print(book_img)
#             print(book_category)

            book_list.append([book_title, book_author, book_publisher, book_isbn, book_img, book_category])

            print('=========>', book_title)

    # 데이터프레임 컬럼명 지정
    colList = ['book_title',  'book_author', 'book_publisher',  'book_ISBN', 'book_img_url', 'book_category']

    # 데이터프레임으로 변환
    df = pd.DataFrame(np.array(book_list), columns=colList)

    return df

In [24]:
CategoryNumber='001001046005'
startPage=1
endPage=2
df = yes24DataReader(CategoryNumber, startPage, endPage)
df
df.to_csv(str(CategoryNumber)+'_'+str(startPage)+' to '+str(endPage)+'.csv', index=False, encoding='CP949')

# Page 1
=========> 안젤리크
=========> 이방인
=========> 이방인
=========> 단순한 열정
=========> 어린 왕자
=========> 브람스를 좋아하세요...
=========> 왜 나는 너를 사랑하는가
=========> 페스트
=========> 자기 앞의 생
=========> 나무를 심은 사람
=========> 초판본 어린왕자
=========> [예스리커버] 장자크 상페 특별판 세트
=========> 낭만적 연애와 그 후의 일상
=========> 행성 1
=========> 행성 2
=========> 고리오 영감
=========> 센 강의 이름 모를 여인
=========> 아노말리
=========> 애린 왕자
=========> 어린 왕자
# Page 2
=========> 고양이 1
=========> 잃어버린 시간을 찾아서 1-6 세트
=========> 집착
=========> 당신, 거기 있어줄래요? Seras-tu la?
=========> 탐닉
=========> 고양이 2
=========> 야간비행
=========> 잃어버린 시간을 찾아서 7~13 세트
=========> 초판본 웃는 남자
=========> 레 미제라블 세트
=========> 남자의 자리
=========> 부끄러움
=========> 한 여자
=========> 꾸뻬 씨의 행복 여행
=========> 반항하는 인간
=========> 세월
=========> 브람스를 좋아하세요...
=========> 구토
=========> 패배의 신호
=========> 문명 1


인코딩 방식 확인  


pip install chardet

In [9]:
import chardet

# 파일 경로 설정
file_path = "./001001046005_1 to 2.csv"

# 파일 바이너리 모드로 열기
with open(file_path, 'rb') as f:
    # 파일 내용 읽기
    contents = f.read()

    # 파일 인코딩 탐지
    result = chardet.detect(contents)

    # 탐지된 인코딩 출력
    print(result['encoding'])


utf-8


* 카테고리 리스트

In [10]:
url='http://www.yes24.com/Mall/Main/Book/001?CategoryNumber=001'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')

In [11]:
categorys = soup.select('li.cate2d>a>em')
categoryNums = soup.select('li.cate2d>a')
re.findall('\d+', soup.select('li.cate2d>a')[0].get('href'))[-1]

print(categorys)
print(categoryNums)

[<em class="txt">가정 살림</em>, <em class="txt">건강 취미</em>, <em class="txt">경제 경영</em>, <em class="txt">국어 외국어 사전</em>, <em class="txt">대학교재</em>, <em class="txt">만화/라이트노벨</em>, <em class="txt">사회 정치</em>, <em class="txt">소설/시/희곡</em>, <em class="txt">수험서 자격증</em>, <em class="txt">어린이</em>, <em class="txt">에세이</em>, <em class="txt">여행</em>, <em class="txt">역사</em>, <em class="txt">예술</em>, <em class="txt">유아</em>, <em class="txt">인문</em>, <em class="txt">인물</em>, <em class="txt">자기계발</em>, <em class="txt">자연과학</em>, <em class="txt">잡지</em>, <em class="txt">전집</em>, <em class="txt">종교</em>, <em class="txt">청소년</em>, <em class="txt">IT 모바일</em>, <em class="txt">초등참고서</em>, <em class="txt">중고등참고서</em>]
[<a class="lnk_cate2d" data-submenu-id="mCateLi_000" href="http://www.yes24.com/24/Category/Display/001001001" onclick="setCdCode('001_001')" style=""><em class="txt">가정 살림</em></a>, <a class="lnk_cate2d" data-submenu-id="mCateLi_001" href="http://www.yes24.com/24/Category/Display/001001011" o

In [12]:
categoryList=[]
categoryNumsList=[]
for category in categorys:
    categoryList.append(category.text)

print(categoryList)
print(len(categoryList))

for categoryNum in categoryNums:
    categoryNumsList.append(re.findall('\d+', categoryNum.get('href'))[-1])
print(categoryNumsList)

['가정 살림', '건강 취미', '경제 경영', '국어 외국어 사전', '대학교재', '만화/라이트노벨', '사회 정치', '소설/시/희곡', '수험서 자격증', '어린이', '에세이', '여행', '역사', '예술', '유아', '인문', '인물', '자기계발', '자연과학', '잡지', '전집', '종교', '청소년', 'IT 모바일', '초등참고서', '중고등참고서']
26
['001001001', '001001011', '001001025', '001001004', '001001014', '001001008', '001001022', '001001046', '001001015', '001001016', '001001047', '001001009', '001001010', '001001007', '001001027', '001001019', '001001020', '001001026', '001001002', '001001024', '001001023', '001001021', '001001005', '001001003', '001001044', '001001013']


In [18]:
df=pd.DataFrame({"book_category_name":categoryList})
df.to_csv("E:\source\scraping/yes24category_UTF8.csv", index = False, encoding='UTF8')